In [2]:
import os

In [ ]:
DATA_DIR = '../data'

BRONZE_DIR = os.path.join(DATA_DIR, 'bronze')
SILVER_DIR = os.path.join(DATA_DIR, 'silver')

# Garante que o diretório silver exista
os.makedirs(SILVER_DIR, exist_ok=True)

In [ ]:
# Nomes dos arquivos na camada bronze
EMPRESA_FILE_BRONZE = os.path.join(BRONZE_DIR, 'K3241.K03200Y0.D50809.EMPRECSV')
SOCIO_FILE_BRONZE = os.path.join(BRONZE_DIR, 'K3241.K03200Y0.D50809.SOCIOCSV')

In [ ]:
# Nomes dos arquivos na camada silver
EMPRESA_FILE_SILVER = os.path.join(SILVER_DIR, 'empresas_silver.parquet')
SOCIO_FILE_SILVER = os.path.join(SILVER_DIR, 'socios_silver.parquet')

In [ ]:
# --- Parâmetros de Leitura dos Arquivos (Comum para Receita Federal) ---
# Os arquivos da Receita Federal geralmente são delimitados por ponto e vírgula,
# codificados em latin-1 e não possuem cabeçalho.
READ_PARAMS = {
    'sep': ';',
    'encoding': 'latin-1',
    'header': None,
    'dtype': str # Ler tudo como string inicialmente para evitar erros de tipo
}

In [ ]:
# --- Processamento da Tabela de EMPRESAS ---
print(f"Lendo arquivo de Empresas: {EMPRESA_FILE_BRONZE}")
try:
    df_empresas_raw = pd.read_csv(EMPRESA_FILE_BRONZE, **READ_PARAMS)

    # Mapeamento das colunas brutas para os nomes e tipos desejados na camada Silver
    # Baseado na estrutura comum dos arquivos da Receita Federal e no schema do desafio.
    # Os índices são 0-based.
    # CNPJ BÁSICO (0), CNPJ ORDEM (1), CNPJ DV (2)
    # RAZÃO SOCIAL (3)
    # CÓDIGO NATUREZA JURÍDICA (5)
    # CÓDIGO PORTE EMPRESA (8)
    # CÓDIGO QUALIFICAÇÃO DO RESPONSÁVEL (9)
    # CAPITAL SOCIAL (10)

    # Seleciona e renomeia as colunas
    df_empresas_silver = df_empresas_raw[[0, 1, 2, 3, 5, 8, 9, 10]].copy()
    df_empresas_silver.columns = [
        'cnpj_basico',
        'cnpj_ordem',
        'cnpj_dv',
        'razao_social',
        'natureza_juridica',
        'cod_porte',
        'qualificacao_responsavel',
        'capital_social'
    ]

    # Concatena as partes do CNPJ para formar o CNPJ completo (14 dígitos)
    df_empresas_silver['cnpj'] = (
        df_empresas_silver['cnpj_basico'] +
        df_empresas_silver['cnpj_ordem'] +
        df_empresas_silver['cnpj_dv']
    )

    # Reordena e seleciona as colunas finais para o schema da camada Silver
    # Mantendo 'cnpj_basico' para futuras junções com a tabela de sócios
    df_empresas_silver = df_empresas_silver[[
        'cnpj',
        'cnpj_basico', # Mantido para junção com socios
        'razao_social',
        'natureza_juridica',
        'qualificacao_responsavel',
        'capital_social',
        'cod_porte'
    ]]

    # Conversão de tipos de dados
    # Usar pd.to_numeric com errors='coerce' para tratar valores não numéricos como NaN
    # e depois preencher NaN com um valor padrão ou remover se necessário
    df_empresas_silver['natureza_juridica'] = pd.to_numeric(df_empresas_silver['natureza_juridica'], errors='coerce').fillna(-1).astype(int)
    df_empresas_silver['qualificacao_responsavel'] = pd.to_numeric(df_empresas_silver['qualificacao_responsavel'], errors='coerce').fillna(-1).astype(int)
    df_empresas_silver['capital_social'] = pd.to_numeric(df_empresas_silver['capital_social'].str.replace(',', '.'), errors='coerce').fillna(0.0).astype(float)
    
    # Limpeza de strings: remover espaços em branco extras
    df_empresas_silver['razao_social'] = df_empresas_silver['razao_social'].str.strip()
    df_empresas_silver['cod_porte'] = df_empresas_silver['cod_porte'].str.strip()

    print(f"Tabela de Empresas processada. Total de registros: {len(df_empresas_silver)}")
    print("Primeiras 5 linhas da tabela Empresas (Silver):")
    print(df_empresas_silver.head())
    print("\nTipos de dados da tabela Empresas (Silver):")
    print(df_empresas_silver.dtypes)

    # Salva o DataFrame processado na camada Silver
    df_empresas_silver.to_parquet(EMPRESA_FILE_SILVER, index=False)
    print(f"Tabela de Empresas salva em: {EMPRESA_FILE_SILVER}")

except FileNotFoundError:
    print(f"ERRO: Arquivo de Empresas não encontrado em {EMPRESA_FILE_BRONZE}. Verifique o caminho.")
except Exception as e:
    print(f"ERRO ao processar arquivo de Empresas: {e}")

print("-" * 50)